In [1]:
import glob
#import cv2
import itertools
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import *
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications.imagenet_utils import preprocess_input

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#import cv2
from matplotlib.pyplot import imread
from matplotlib.pyplot import imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB7
import tensorflow as tf
import pandas as pd 
import matplotlib.pyplot as plt
import os


NUM_CLASSES = 2
IMG_SIZE = 224


In [2]:

pwd

'C:\\Users\\shaya\\assignment 1 PDC'

# ***DATA LOADING AND PRE-PROCESSING***

In [13]:
import os
images = []
labels =[]
for sub_dir in os.listdir('Training'):
  image_list=os.listdir(os.path.join('Training',sub_dir))  #list of all image names in the directory
  image_list = list(map(lambda x:os.path.join(sub_dir,x),image_list))
  images.extend(image_list)
  labels.extend([sub_dir]*len(image_list)) 

NotADirectoryError: [WinError 267] The directory name is invalid: 'Forest Fire Dataset\\efficientattentionet.ipynb'

In [ ]:
df = pd.DataFrame({"Images":images,"Labels":labels})      
df = df.sample(frac=1).reset_index(drop=True) # To shuffle the data 
df_train = df.head(10000)
df_test = df.tail(3000)

In [ ]:
train_data = df_train.groupby('Labels').count()
test_data = df_test.groupby('Labels').count()

In [ ]:
train_data

In [8]:
test_data

,Images
Labels,
fire,760
nofire,760


In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=50,
    validation_split=0.3,
    )

In [8]:
train_generator=train_datagen.flow_from_dataframe(
dataframe=df_train,
directory='/kaggle/input/flamedataset/train',
x_col="Images",
y_col="Labels",
subset='training',
batch_size=16,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))


valid_generator=train_datagen.flow_from_dataframe(
dataframe=df_train,
directory='/kaggle/input/flamedataset/train',
x_col="Images",
y_col="Labels",
subset='validation',
batch_size=16,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

Found 7000 validated image filenames belonging to 2 classes.
Found 3000 validated image filenames belonging to 2 classes.


In [9]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes), 
    y=train_generator.classes
)
train_class_weights = dict(enumerate(class_weights))
train_class_weights

{0: 0.7877560207067297, 1: 1.368791552600704}

In [10]:
pip install keras-tuner

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.6 MB/s eta 0:00:0000:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.10.1 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.61.0 which is incompatible.
tfx-bsl 1.10.1 requires pyarrow<7,>=6, but you have pyarrow 9.0.0 which is incompatible.
tensorflow-transform 1.10.1 requires pyarrow<7,>=6, but you have pyarrow 9.0.0 which is incompatible.
tensorflow-serving-api 2.10.0 requires tensorflow<3,>=2.10.0, but you have tensorflow 2.9.2 which is incompatible.
ortools 9.5.2237 requires protobuf>=4.21.5, b

Note: you may need to restart the kernel to use updated packages.


In [11]:
import tensorflow as tf
from kerastuner.tuners import BayesianOptimization
import keras_tuner as kt
hp = kt.HyperParameters()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


# ***MODEL BUILDING***

In [12]:
# ECA BLOCK
def ECA(x):
    k_size = 3 
    squeeze = tf.reduce_mean(x,(2,3),keepdims=False)
    squeeze = tf.expand_dims(squeeze, axis=1)
    attn = layers.Conv1D(filters=1,
    kernel_size=k_size,
    padding='same',
    kernel_initializer='random_normal',
    use_bias=False)(squeeze)

    attn = tf.expand_dims(tf.transpose(attn, [0, 2, 1]), 3)
    attn = tf.math.sigmoid(attn)
    scale = x * attn
    return x * attn

In [13]:
def resblock(inputs, filters, strides):
    
    y = inputs # Shortcut path
    
    # Main path
    x = tf.keras.layers.Conv2D(
        filters=filters,
        kernel_size=3,
        strides=strides,
        padding='same',
    )(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv2D(
        filters=filters,
        kernel_size=3,
        strides=1,
        padding='same',
    )(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # Fit shortcut path dimenstions
    if strides > 1:
        y = tf.keras.layers.Conv2D(
        filters=filters,
        kernel_size=3,
        strides=strides,
        padding='same',
        )(y)
        y = tf.keras.layers.BatchNormalization()(y)

    # Concatenate paths
    x = tf.keras.layers.Add()([x, y])
    x = tf.keras.layers.ReLU()(x)

    return x

# ***HYPER PARAM TUNING***

In [14]:
def build_model(hp):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = inputs
    model = EfficientNetB7(include_top=False, input_tensor=x, weights="imagenet")
    
    for layer in model.layers:
        if layer.name not in ['block7d_project_conv']:
            layer.trainable = False

    # Rebuild top
#     x = tf.keras.layers.ReLU()(model.output)
#     filters_res=hp.Int('Res_filter', min_value=32, max_value=512, step=32)
#     x = resblock(x, filters_res,  2)
#     x = resblock(x, filters_res,  1)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = layers.Flatten()(x)
    
    

    
    # ECA-Net

    x1 = layers.Resizing(IMG_SIZE, IMG_SIZE)(inputs)
    x2 = layers.Rescaling(1./255)(x1)
    x3 = layers.Conv2D(filters=hp.Int('conv_1_filter', min_value=16, max_value=32, step=5), kernel_size=(3, 3), activation='relu', 
                      padding = 'same', input_shape=(IMG_SIZE,IMG_SIZE,3))(x2)
    x4 = ECA(x3)
    x5 = layers.BatchNormalization(axis=-1)(x4)
    x6 = layers.MaxPool2D(pool_size=(2, 2))(x5)
    x7 = layers.Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=10), kernel_size=(3, 3), 
                      activation='relu', padding = 'same')(x6)
    x8 = ECA(x7)
    x9 = layers.BatchNormalization(axis=-1)(x8)
    x10 = layers.MaxPool2D(pool_size=(2, 2))(x9)
    x11 = layers.Conv2D(filters=hp.Int('conv_3_filter', min_value=64, max_value=128, step=15), kernel_size=(3, 3), 
                        activation='relu', padding = 'same')(x10)

    x12 = ECA(x11)
    x13 = layers.BatchNormalization(axis=-1)(x12)
    x14 = layers.MaxPool2D(pool_size=(2, 2))(x13)
    x15 = layers.Conv2D(filters=hp.Int('conv_4_filter', min_value=128, max_value=256, step=20), kernel_size=(3, 3), 
                          activation='relu', padding = 'same')(x14)
    x16 = ECA(x15)
    x17 = layers.BatchNormalization(axis=-1)(x16)
    x18 = layers.MaxPool2D(pool_size=(2, 2))(x17)
    y1 = layers.Flatten()(x18)
    concat = Concatenate()([y1,x])
    dense1  = layers.Dense(units=hp.Int('dense_units_1', min_value=128, max_value=512, step=64), activation='relu')(concat)
    dropout1= Dropout(hp.Float('dropout', 0.25, 0.4, step=0.05))(dense1)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(dropout1)

    # Compile
       
    model = tf.keras.Model(inputs, outputs, name="EfficientAttentioNet")
    model.compile(
         optimizer=keras.optimizers.Adam(
         hp.Choice('learning_rate', values=[1e-4, 1e-5])),
         loss='categorical_crossentropy',
         metrics=['accuracy'])
    
    return model

In [15]:
tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='my_dir',
    project_name='Optimization')

258076736/258076736 [==============================] - 8s 0us/step


In [16]:
tuner.search(train_generator, epochs=5, validation_data=valid_generator,class_weight=train_class_weights)

# Print the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Optimal number of filter in the first Conv layer: {best_hps.get('conv_1_filter')}")
print(f"Optimal number of filter in the second Conv layer: {best_hps.get('conv_2_filter')}")
print(f"Optimal number of filter in the Third Conv layer: {best_hps.get('conv_3_filter')}")
print(f"Optimal number of filter in the Fourth Conv layer: {best_hps.get('conv_4_filter')}")
print(f"Optimal number of units in the first dense layer: {best_hps.get('dense_units_1')}")
print(f"Optimal number of units in the dropout layer: {best_hps.get('dropout')}")
print(f"Optimal learning rate: {best_hps.get('learning_rate')}")


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
21                |?                 |conv_1_filter
62                |?                 |conv_2_filter
94                |?                 |conv_3_filter
148               |?                 |conv_4_filter
448               |?                 |dense_units_1
0.35              |?                 |dropout
0.0001            |?                 |learning_rate

Epoch 1/5
319/438 [====================>.........] - ETA: 28s - loss: 0.3747 - accuracy: 0.9134

In [ ]:
def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = inputs
    model = EfficientNetB7(include_top=False, input_tensor=x, weights="imagenet")

    for layer in model.layers:
        if layer.name not in ['block7d_project_conv']:
            layer.trainable = False

        if layer.trainable:
            print(layer.name)

    # Rebuild top
    x = layers.BatchNormalization()(model.output)
    x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = layers.Flatten()(x)
    
    

    
    # ECA-Net

    x1 = layers.Resizing(IMG_SIZE, IMG_SIZE)(inputs)
    x2 = layers.Rescaling(1./255)(x1)
    x3 = layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', 
                      padding = 'same', input_shape=(IMG_SIZE,IMG_SIZE,3))(x2)
    x4 = ECA(x3)
    x5 = layers.BatchNormalization(axis=-1)(x4)
    x6 = layers.MaxPool2D(pool_size=(2, 2))(x5)
    x7 = layers.Conv2D(filters=64, kernel_size=(3, 3), 
                      activation='relu', padding = 'same')(x6)
    x8 = ECA(x7)
    x9 = layers.BatchNormalization(axis=-1)(x8)
    x10 = layers.MaxPool2D(pool_size=(2, 2))(x9)
    x11 = layers.Conv2D(filters=128, kernel_size=(3, 3), 
                        activation='relu', padding = 'same')(x10)

    x12 = ECA(x11)
    x13 = layers.BatchNormalization(axis=-1)(x12)
    x14 = layers.MaxPool2D(pool_size=(2, 2))(x13)
    x15 = layers.Conv2D(filters=256, kernel_size=(3, 3), 
                          activation='relu', padding = 'same')(x14)
    x16 = ECA(x15)
    x17 = layers.BatchNormalization(axis=-1)(x16)
    x18 = layers.MaxPool2D(pool_size=(2, 2))(x17)
    y1 = layers.Flatten()(x18)
    concat = Concatenate()([y1,x])
    dense1  = layers.Dense(512, activation='relu')(concat)
    dropout1= Dropout(0.4)(dense1)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(dropout1)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientAttentioNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5) 
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=['Accuracy']
    )
    return model

In [ ]:
model = build_model(num_classes=NUM_CLASSES)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model 
plot_model(model, to_file="my_model.png", show_shapes=True)

In [ ]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.figure(figsize=(20,5))
    plt.subplot(1,2,1)
    plt.plot(hist.history['Accuracy'])
    plt.plot(hist.history['val_Accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='lower right')

    plt.subplot(1,2,2)
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=6)

In [ ]:
epochs = 50 
hist = model.fit_generator( generator=train_generator,
                            validation_data=valid_generator,
                            epochs=epochs,
                            class_weight=train_class_weights,
                            callbacks=[callback],
                            verbose=1
)

In [ ]:
plot_hist(hist)  # took 2h:41m to complete

# ***EVALUATION***

In [ ]:
score = model.evaluate(valid_generator, verbose = 0) 
print('Validation loss:', score[0]) 
print('Validation accuracy:', score[1])

In [ ]:
test_generator=train_datagen.flow_from_dataframe(
dataframe=df_test,
directory='/kaggle/input/flamedataset/train',
x_col="Images",
y_col="Labels",
batch_size=16,
seed=42,
shuffle=False,
class_mode="categorical",
target_size=(224,224))

In [ ]:
test_score = model.evaluate(test_generator, verbose = 0) 
print('Test loss:', test_score[0]) 
print('Test accuracy:', test_score[1])

In [ ]:
target_names = []

for key in train_generator.class_indices:
    target_names.append(key)
    
print(target_names)

# ***MATRICES***

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(15,15))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

***CONFUSION MATRIX***

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
# Y_pred = model.predict(x_test,batch_size=16)
Y_pred = model.predict(test_generator,batch_size=16)
y_pred = np.argmax(Y_pred, axis=1)


# labels = np.argmax(y_test,axis=-1)

print('Confusion Matrix')
cm = confusion_matrix(test_generator.classes, y_pred)
# cm = confusion_matrix(labels, y_pred)
plot_confusion_matrix(cm, target_names, title='Confusion Matrix', normalize=False)




***CLASSIFICATION REPORT***

In [ ]:
print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

***ROC-AUC CURVE***

In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc


x_test, y_test = next(test_generator)

n_classes = 2

pred_Y = model.predict(x_test, batch_size = 16, verbose = True)
# Plot linewidth.
lw = 2

# Compute ROC curve and ROC area for each class

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], pred_Y[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    # Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), pred_Y.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
# Plot of a ROC curve for a specific class
for i in range(n_classes):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
fig = plt.figure(figsize=(12, 8))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))


plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi Class Receiver operating curve')
plt.legend(loc="lower right")
sns.despine()
plt.show()

# ***GRAD-CAM***

In [ ]:
class_indices = test_generator.class_indices
indices = {v:k for k,v in class_indices.items()}
filenames = test_generator.filenames

In [ ]:
val_df = pd.DataFrame()
val_df['filename'] = filenames
val_df['actual'] = test_generator.classes
val_df['predicted'] = y_pred
val_df['actual'] = val_df['actual'].apply(lambda x: indices[x])
val_df['predicted'] = val_df['predicted'].apply(lambda x: indices[x])
val_df.loc[val_df['actual']==val_df['predicted'],'Same'] = True
val_df.loc[val_df['actual']!=val_df['predicted'],'Same'] = False

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
val_df = val_df.sample(frac=1).reset_index(drop=True)

In [ ]:
val_df.head(10)

In [ ]:
def readImage(path):
    img = load_img(path,color_mode='rgb',target_size=(224,224))
    img = img_to_array(img)
    img = img/255.
    
    return img

def display_images(temp_df):
    temp_df = temp_df.reset_index(drop=True)
    plt.figure(figsize = (25 , 25))
    n = 0
    for i in range(3):
        n+=1
        plt.subplot(3 , 3, n)
        plt.subplots_adjust(hspace = 0.5 , wspace = 0.3)
        image = readImage(f"/kaggle/input/flamedataset/train/{temp_df.filename[i]}")
        plt.imshow(image)
        plt.title(f'A: {temp_df.actual[i]} P: {temp_df.predicted[i]}')

In [ ]:
from tensorflow.keras.models import Sequential, Model
import matplotlib.cm as cm

def make_gradcam_heatmap(img_array, model1, pred_index=None):
    
    grad_model = Model(inputs=model1.inputs, outputs=[model1.get_layer("conv2d_15").output, model1.output]) #model1.get_layer("conv2d_4").output #model1.layers[-17].output

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy(), preds 

In [ ]:
def gradCAMImage(image):
    path = f"/kaggle/input/flamedataset/train/{image}"
    img = readImage(path)
    img = np.expand_dims(img,axis=0)
    heatmap,preds = make_gradcam_heatmap(img,model)

    img = load_img(path)
    img = img_to_array(img)
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * 0.8 + img
    # superimposed_img = 0.8 + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)
    
    return superimposed_img

In [ ]:
def gradcam_of_images(correct_class):
    grad_images = []
    title = []
    temp_df = val_df[val_df['Same']==correct_class]
    temp_df = temp_df.reset_index(drop=True)
    for i in range(15):
        image = temp_df.filename[i]
        grad_image = gradCAMImage(image)
        grad_images.append(grad_image)
        title.append(f"A: {temp_df.actual[i]} P: {temp_df.predicted[i]}")

    return grad_images, title

In [ ]:
correctly_classified, c_titles = gradcam_of_images(correct_class=True)
misclassified, m_titles = gradcam_of_images(correct_class=False)

In [ ]:
def display_heatmaps(classified_images,titles):
    plt.figure(figsize = (25 ,25))
    n = 0
    for i in range(3):
        n+=1
        plt.subplot(3 ,3 , n)
        plt.subplots_adjust(hspace = 0.3 , wspace = 0.3)
        plt.imshow(classified_images[i])
        plt.title(titles[i])
#         plt.imshow(val_df[val_df['Same']==True])
    plt.show()

In [ ]:
display_images(val_df[val_df['Same']==True])

In [ ]:
display_heatmaps(correctly_classified,c_titles)

In [ ]:
display_images(val_df[val_df['Same']!=True])

In [ ]:
display_heatmaps(misclassified,m_titles)

# ***LIME***

In [ ]:
pip install lime

In [ ]:
pip install scikit-image

In [ ]:
import lime
import lime.lime_tabular
import skimage
import numpy as np

In [ ]:
from skimage import io
from tensorflow.keras.preprocessing import image

# url = '/kaggle/input/flamedataset/train/Fire/resized_frame0.jpg'
# url = '/kaggle/input/flamedataset/train/Fire/resized_frame11264.jpg'
url = '/kaggle/input/flamedataset/train/Fire/resized_frame7045.jpg'

def read_and_transform_img(url):

    img = skimage.io.imread(url)
    img = skimage.transform.resize(img, (224,224))
    
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)

    return img

images = read_and_transform_img(url)

In [ ]:
# LIME has one explainer for all the models
from lime import lime_image
explainer = lime_image.LimeImageExplainer(random_state=42)

In [ ]:
explanation = explainer.explain_instance(images[0].astype('double'), model.predict,  
                                         top_labels=5, hide_color=0, num_samples=1000)

In [ ]:
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

temp_1, mask_1 = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1/2+0.5, mask_1))
ax2.imshow(mark_boundaries(temp_2/2, mask_2))
ax1.axis('off')
ax2.axis('off')

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import image as mpimg

plt.title("Test Image")
plt.xlabel("X pixel scaling")
plt.ylabel("Y pixels scaling")

image = mpimg.imread(url)
plt.imshow(image)
plt.show()

In [ ]:
import tensorflow as tf
import numpy as np
import cv2

# Define CAM class
class CAM:
    def __init__(self, model, last_conv_layer_name):
        self.model = model
        self.last_conv_layer_name = last_conv_layer_name
        self.grad_model = tf.keras.models.Model([self.model.inputs], [self.model.get_layer(self.last_conv_layer_name).output, self.model.output])
        
    def compute_heatmap(self, image):
        with tf.GradientTape() as tape:
            conv_output, predictions = self.grad_model(image)
            class_idx = tf.argmax(predictions[0])
            loss = predictions[:, class_idx]
            
        output = conv_output[0]
        grads = tape.gradient(loss, conv_output)[0]
        weights = tf.reduce_mean(grads, axis=(0, 1))
        heatmap = tf.reduce_sum(tf.multiply(weights, output), axis=-1)
        heatmap = np.maximum(heatmap, 0)
        heatmap /= np.max(heatmap)
        return heatmap

# Load input image
image = cv2.imread('/kaggle/input/flamedataset/train/Fire/resized_frame7045.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))
image = img_to_array(image)
image = np.expand_dims(image, axis=0)


# Define CAM model
cam_model = CAM(model, 'conv2d_15')

# Compute heatmap
heatmap = cam_model.compute_heatmap(image)

# Rescale heatmap to the range [0, 255]
heatmap = heatmap * 255.0
heatmap = cv2.resize(heatmap, (224, 224))

# Apply colormap to heatmap
heatmap = cv2.applyColorMap(np.uint8(heatmap), cv2.COLORMAP_JET)

# Overlay heatmap on input image
result = heatmap * 0.5 + image[0] * 0.5
result = tf.keras.preprocessing.image.array_to_img(result)

from matplotlib import pyplot as plt
from matplotlib import image as mpimg

plt.title("Test Image")
plt.xlabel("X pixel scaling")
plt.ylabel("Y pixels scaling")

plt.imshow(result)
plt.show()